In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2 as cv
import re
import requests

from sklearn.metrics import confusion_matrix,  multilabel_confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from keras.layers import BatchNormalization, Activation, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

In [74]:
import pandas as pd

data = pd.read_csv('/kaggle/input/fashion-product-images-small/styles.csv', usecols = ['id', 'gender', 'baseColour']) 

In [75]:
data.head()

,id,gender,baseColour
0,15970,Men,Navy Blue
1,39386,Men,Blue
2,59263,Women,Silver
3,21379,Men,Black
4,53759,Men,Grey


In [76]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44446 entries, 0 to 44445
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          44446 non-null  int64 
 1   gender      44446 non-null  object
 2   baseColour  44431 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [77]:
data = data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44431 entries, 0 to 44445
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          44431 non-null  int64 
 1   gender      44431 non-null  object
 2   baseColour  44431 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [78]:
data_encoded = pd.get_dummies(data, columns=['gender', 'baseColour'], dtype = 'int')

In [79]:
data_encoded.head()

,id,gender_Boys,gender_Girls,gender_Men,gender_Unisex,gender_Women,baseColour_Beige,baseColour_Black,baseColour_Blue,baseColour_Bronze,...,baseColour_Sea Green,baseColour_Silver,baseColour_Skin,baseColour_Steel,baseColour_Tan,baseColour_Taupe,baseColour_Teal,baseColour_Turquoise Blue,baseColour_White,baseColour_Yellow
0,15970,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,39386,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,59263,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,21379,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,53759,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
data_encoded.shape[0]

44431

In [81]:
data_encoded.shape[1]

52

In [82]:
from tqdm import tqdm
from keras.preprocessing import image
from tensorflow.keras.applications import ResNet152V2, InceptionResNetV2, EfficientNetV2S
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input as efficientnet_preprocess
from sklearn.model_selection import train_test_split
import os


dataset_path = '/kaggle/input/fashion-product-images-small/images/'
target_size = (224, 224)
batch_size = 32 


data_encoded['filename'] = data_encoded['id'].astype(str) + '.jpg'
y_columns = data_encoded.columns.drop(['id', 'filename'])  


train_df, test_df = train_test_split(data_encoded, test_size=0.3, random_state=20)


In [83]:
y_columns

Index(['gender_Boys', 'gender_Girls', 'gender_Men', 'gender_Unisex',
       'gender_Women', 'baseColour_Beige', 'baseColour_Black',
       'baseColour_Blue', 'baseColour_Bronze', 'baseColour_Brown',
       'baseColour_Burgundy', 'baseColour_Charcoal', 'baseColour_Coffee Brown',
       'baseColour_Copper', 'baseColour_Cream', 'baseColour_Fluorescent Green',
       'baseColour_Gold', 'baseColour_Green', 'baseColour_Grey',
       'baseColour_Grey Melange', 'baseColour_Khaki', 'baseColour_Lavender',
       'baseColour_Lime Green', 'baseColour_Magenta', 'baseColour_Maroon',
       'baseColour_Mauve', 'baseColour_Metallic', 'baseColour_Multi',
       'baseColour_Mushroom Brown', 'baseColour_Mustard',
       'baseColour_Navy Blue', 'baseColour_Nude', 'baseColour_Off White',
       'baseColour_Olive', 'baseColour_Orange', 'baseColour_Peach',
       'baseColour_Pink', 'baseColour_Purple', 'baseColour_Red',
       'baseColour_Rose', 'baseColour_Rust', 'baseColour_Sea Green',
       'baseColour_S

In [84]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(
    preprocessing_function=efficientnet_preprocess,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

test_gen = ImageDataGenerator(
    preprocessing_function=efficientnet_preprocess
)

train_generator = train_gen.flow_from_dataframe(
    dataframe=train_df,
    directory=dataset_path,
    x_col='filename',
    y_col=list(y_columns),
    target_size=target_size,
    class_mode='raw',  
    batch_size=batch_size,
    shuffle=True
)


test_generator = test_gen.flow_from_dataframe(
    dataframe=test_df,
    directory=dataset_path,
    x_col='filename',
    y_col=list(y_columns),
    target_size=target_size,
    class_mode='raw',
    batch_size=batch_size,
    shuffle=False
)

Found 31097 validated image filenames.


/usr/local/lib/python3.10/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 4 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


Found 13329 validated image filenames.


/usr/local/lib/python3.10/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 1 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


In [85]:

import tensorflow as tf
from tensorflow.keras.applications import ResNet152V2, InceptionResNetV2, EfficientNetV2S
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input as efficientnet_preprocess
from tensorflow.keras import layers, models

def efficeint_builder(): 
  base_model = EfficientNetV2S(
      weights='imagenet',
      include_top=False,
      input_shape=target_size + (3,),
      pooling = 'avg'
  )
  base_model.trainable = False

  model = models.Sequential([
      base_model,
      layers.Dense(128, activation='relu'),  
      layers.Dropout(0.3),
      layers.Dense(51, activation='sigmoid')
  ])

  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
      loss='binary_crossentropy',
      metrics=['accuracy']
  )

  return model


In [86]:
model = efficeint_builder()
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetv2-s (Functional)        │ (None, 1280)                │      20,331,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 128)                 │         163,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 51)                  │           6,579 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,501,907 (78.21 MB)

 Trainable params: 170,547 (666.20 KB)

 Non-trainable params: 20,331,360 (77.56 MB)

In [87]:
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True , verbose=1)]

history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=10,
    verbose = 1
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


972/972 ━━━━━━━━━━━━━━━━━━━━ 445s 414ms/step - accuracy: 0.6533 - loss: 0.1334 - val_accuracy: 0.8091 - val_loss: 0.0653
Epoch 2/10
972/972 ━━━━━━━━━━━━━━━━━━━━ 372s 380ms/step - accuracy: 0.7718 - loss: 0.0692 - val_accuracy: 0.7977 - val_loss: 0.0599
Epoch 3/10
972/972 ━━━━━━━━━━━━━━━━━━━━ 372s 380ms/step - accuracy: 0.7777 - loss: 0.0646 - val_accuracy: 0.8008 - val_loss: 0.0574
Epoch 4/10
972/972 ━━━━━━━━━━━━━━━━━━━━ 370s 378ms/step - accuracy: 0.7780 - loss: 0.0622 - val_accuracy: 0.8229 - val_loss: 0.0567
Epoch 5/10
972/972 ━━━━━━━━━━━━━━━━━━━━ 374s 382ms/step - accuracy: 0.7849 - loss: 0.0606 - val_accuracy: 0.8097 - val_loss: 0.0552
Epoch 6/10
972/972 ━━━━━━━━━━━━━━━━━━━━ 373s 382ms/step - accuracy: 0.7846 - loss: 0.0587 - val_accuracy: 0.8177 - val_loss: 0.0548
Epoch 7/10
972/972 ━━━━━━━━━━━━━━━━━━━━ 374s 382ms/step - accuracy: 0.7951 - loss: 0.0578 - val_accuracy: 0.8125 - val_loss: 0.0541
Epoch 8/10
972/972 ━━━━━━━━━━━━━━━━━━━━ 372s 380ms/step - accuracy: 0.7923 - loss: 0.05

In [91]:
# fine tuning on a subset
subset_size = int(0.4 * len(data_encoded))


subset_df = data_encoded.sample(n=subset_size, random_state=42)

train_df, test_df = train_test_split(subset_df, test_size=0.1, random_state=20)

In [92]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(
    preprocessing_function=efficientnet_preprocess,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

test_gen = ImageDataGenerator(
    preprocessing_function=efficientnet_preprocess
)

train_generator = train_gen.flow_from_dataframe(
    dataframe=train_df,
    directory=dataset_path,
    x_col='filename',
    y_col=list(y_columns),
    target_size=target_size,
    class_mode='raw',  
    batch_size=batch_size,
    shuffle=True
)


test_generator = test_gen.flow_from_dataframe(
    dataframe=test_df,
    directory=dataset_path,
    x_col='filename',
    y_col=list(y_columns),
    target_size=target_size,
    class_mode='raw',
    batch_size=batch_size,
    shuffle=False
)

Found 15992 validated image filenames.


/usr/local/lib/python3.10/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 2 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


Found 1778 validated image filenames.


In [93]:
base_model = model.layers[0]  
base_model.trainable = True  

for layer in base_model.layers[:int(len(base_model.layers)*0.8)]:  
    layer.trainable = False

In [94]:
  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
      loss='binary_crossentropy',
      metrics=['accuracy']
  )

In [95]:
history_2 = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=10,
    verbose = 1,
    callbacks = callbacks
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


500/500 ━━━━━━━━━━━━━━━━━━━━ 288s 450ms/step - accuracy: 0.7643 - loss: 0.0767 - val_accuracy: 0.7998 - val_loss: 0.0534
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 178s 350ms/step - accuracy: 0.7739 - loss: 0.0614 - val_accuracy: 0.8127 - val_loss: 0.0514
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 180s 356ms/step - accuracy: 0.7746 - loss: 0.0594 - val_accuracy: 0.8060 - val_loss: 0.0500
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 179s 352ms/step - accuracy: 0.7824 - loss: 0.0567 - val_accuracy: 0.8110 - val_loss: 0.0491
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 180s 354ms/step - accuracy: 0.7883 - loss: 0.0554 - val_accuracy: 0.8150 - val_loss: 0.0485
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 179s 352ms/step - accuracy: 0.7921 - loss: 0.0539 - val_accuracy: 0.8245 - val_loss: 0.0479
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 180s 355ms/step - accuracy: 0.7937 - loss: 0.0533 - val_accuracy: 0.8155 - val_loss: 0.0473
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 179s 353ms/step - accuracy: 0.7908 - loss: 0.05

In [96]:
model.save('fashion_v2.keras')

In [ ]:
label_names = list(y_columns)

In [37]:
image_path = '/kaggle/input/fashion-product-images-small/images/14856.jpg'

In [31]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array




def predict_top_labels(model, image_path, target_size=(224, 224)):
    image = load_img(image_path, target_size=target_size)
    image_array = img_to_array(image)
    image_array = np.expand_dims(image_array, axis=0)
    image_array = efficientnet_preprocess(image_array)

    predictions = model.predict(image_array)[0]
    
    
    gender_predictions = [(label, predictions[i])
                          for i, label in enumerate(label_names)
                          if label.startswith("gender_")]
    top_gender = max(gender_predictions, key=lambda x: x[1]) if gender_predictions else None
    
   
    colour_predictions = [(label, predictions[i])
                          for i, label in enumerate(label_names)
                          if label.startswith("baseColour_")]
    top_colour = max(colour_predictions, key=lambda x: x[1]) if colour_predictions else None
    
    return top_gender, top_colour

In [38]:
print(predict_top_labels(model, image_path))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
(('gender_Women', 0.94500405), ('baseColour_Brown', 0.30091295))
